# This notebook demonstrates how to use the Pace utility

This utility can be used to limit the rate of API requests to external endpoints. 

In [ ]:
import asyncio
import random
import threading
import time

from IPython import display
from trulens.core.utils.pace import Pace

In [ ]:
# Create a pace instance with 2 second per period and 20 marks per second. The
# average of 20 marks per second will be maintained across any 2 second period
# but this makes it possible for an initial burst of 20 marks immediately. This
# is due to the assumption that there were no marks before the process started.

# If seconds_per_period is increased, a larger burst of marks will be possible
# before the average marks per second since the start of the process stabilizes.
# A larger burst also means there will be a delay until the next period before
# marks can return again. A "burstiness" warning is issue the first time a delay
# longer than half of the seconds_per_period is encountered.

p = Pace(seconds_per_period=2, marks_per_second=20)

In [ ]:
# start time and counter
st = time.time()
count = 0

while True:
    # Mark and increment counter. Calls to mark will block to maintain pace.
    p.mark()
    count += 1

    et = time.time()
    display.clear_output(wait=True)

    # Show stats of the marks rate since the start of this cell.
    print(f"""
Elapsed time: {et - st}
Marks count: {count}
Marks per second: {count / (et - st)}
""")

# Pace across Threads

The pacing should be maintained even if a single Pace instance is used across
multiple threads.

In [ ]:
num_threads = 10
count = 0


# Create a function to run in each thread and update the count for each mark:
def marker():
    global count

    while True:
        # Mark and increment counter. Calls to mark will block to maintain pace.
        p.mark()
        count += 1

        # Add a bit of sleep to simulate some work.
        time.sleep(random.random() / 100.0)


# Start time.
st = time.time()

# Start the threads.
for i in range(num_threads):
    t = threading.Thread(target=marker)
    t.start()

while True:
    # Report count stats every second.
    time.sleep(1)

    display.clear_output(wait=True)

    et = time.time()

    # Show stats of the marks rate since the start of this cell.
    print(f"""
Elapsed time: {et - st}
Marks count: {count}
Marks per second: {count / (et - st)}
""")

# Pace in Async Tasks

Pace can also be maintained when using asynchronous tasks. For this, the `amark`
method must be used and awaited.

In [ ]:
num_tasks = 10
count = 0


# Create a function to run in each task and update the count for each mark:
async def async_marker():
    global count

    while True:
        # Mark and increment counter. Calls to amark will block to maintain pace.
        await p.amark()
        count += 1

        # Add a bit of sleep to simulate some work.
        await asyncio.sleep(random.random() / 100.0)


# Start time.
st = time.time()

loop = asyncio.get_event_loop()

# Start the threads.
for i in range(num_tasks):
    task = loop.create_task(async_marker())

while True:
    # Report count stats every second.

    await asyncio.sleep(1)

    display.clear_output(wait=True)

    et = time.time()

    # Show stats of the marks rate since the start of this cell.
    print(f"""
max_marks: {p.max_marks}
mark_expirations: {p.mark_expirations}
len(mark_expirations): {len(p.mark_expirations)}
Elapsed time: {et - st}
Marks count: {count}
Marks per second: {count / (et - st)}
""")